# Vehicle Make Model Recognition

## Objective
Objective of this project is to train a classifier model - specifically, a model that receives an image of a vehicle as input and then outputs a predicted label indicating the vehicle's **make and model**.

Previously, we used the ```data_split.ipynb``` to split the dataset into train, test, val folders.

## Task Overview
This notebook contains a basic ResNet50 model for classifying the vehicles. You may adjust the training parameters to finetune the model, including but not limited to the **epochs** and **learning_rate**.

You may also explore other types of classification models such as VGG, Inception, EfficientNet, etc. More details about models you can explore can be found here: https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16  
 

## To-dos
1. Try changing the learning rate and number of epochs to see how it affects the model
2. Explore other classification models
3. Try out other parameters in the ResNet50 model
4. Improve the model through techniques of balancing the imbalanced dataset
5. Plot ROC metrics for the training

## Training

In [ ]:
import tensorflow as tf

In [ ]:
# Load image from directory
root = 'data_split' # Insert path of root folder here
TRAINING_FOLDER = root + "/TRAIN/"
VALIDATION_FOLDER = root + "/VAL/"
NUMBER_OF_CLASSES = 5

In [ ]:
all_classes = sorted(['VOLVO_FM12', 'HINO_FN2P', 'SINOTRUK_A7', 'HINO_SH1E', 'MITSUBISHI_FP517'])


In [ ]:
# Training parameters to change
epochs = 10
learning_rate = 0.0005

In [ ]:
# Preparation of dataset

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=TRAINING_FOLDER,
    seed=123,
    image_size=(224, 224),
    batch_size=2,
    label_mode='categorical'
)
print(train_ds.class_names)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=VALIDATION_FOLDER,
    seed=123,
    image_size=(224, 224),
    batch_size=2,
    label_mode='categorical'
)

In [ ]:
# Define model
# Refer here for other parameters to be added: 
# https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50

base_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(224, 224, 3),
    weights='imagenet'
)
base_model.trainable = False

In [ ]:

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(NUMBER_OF_CLASSES, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=tf.keras.metrics.categorical_accuracy)
print(model.summary())

In [ ]:
# Training
history = model.fit(train_ds,
          epochs=epochs,
          validation_data=val_ds)

In [ ]:
from matplotlib import pyplot as plt

def plot_metrics(history):
  metrics = ['loss', 'categorical_accuracy']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color='blue', label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color='blue', linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

plot_metrics(history)

In [ ]:
# Output model
model.save("my_model")

## Evaluation

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import os
from sklearn.metrics import classification_report

In [ ]:
# load images from directory
TEST_FOLDER = "data_split/TEST/"

In [ ]:
# load model
MODEL = 'my_model'
model = tf.keras.models.load_model(MODEL)

In [ ]:
all_classes = sorted(['VOLVO_FM12', 'HINO_FN2P', 'SINOTRUK_A7', 'HINO_SH1E', 'MITSUBISHI_FP517'])


In [ ]:
# Pred
preds = []
actual = []

for i in range(len(all_classes)):
    category = all_classes[i]
    class_folder = os.path.join(TEST_FOLDER, category)

    img_array = []

    for file in os.listdir(class_folder):
        img_path = os.path.join(TEST_FOLDER, category, file)

        img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
        new_img = tf.keras.utils.img_to_array(img)

        if len(img_array) == 0:
            img_array = np.array([new_img])
        else:
            img_array = np.vstack((img_array, [new_img]))
        actual.append(i)
    

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions)
    pred_class = np.argmax(score, axis=1)
    preds += pred_class.flatten().tolist()

In [ ]:
print(classification_report(actual, preds, target_names=all_classes))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
cm = confusion_matrix(actual, preds)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp = ConfusionMatrixDisplay(confusion_matrix=cmn, display_labels=all_classes)
disp.plot()
plt.show()

# Prediction

In [ ]:
# load model
MODEL = 'my_model'
model = tf.keras.models.load_model(MODEL)

In [ ]:
TEST_IMAGE = TEST_FOLDER + 'HINO_FN2P/group1_9_0_42_29.jpeg'
img = tf.keras.utils.load_img(TEST_IMAGE, target_size=(224, 224))

In [ ]:
# prediction
img_array = tf.keras.utils.img_to_array(img)
img_array = np.array([img_array])

In [ ]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
pred_class = all_classes[np.argmax(score)]

In [ ]:
from IPython.display import Image, display
print("Predicted class: ", pred_class)
print('Actual class: ', TEST_IMAGE.split('/')[-2])
print("Probability: ", end='')
tf.print(tf.reduce_max(score))
display(Image(filename=TEST_IMAGE))